# Pulsar Timing with PINT

![test.png](./images/gravitational_wave_pulsars.png)

## Introduction




In this exercise, you will see how different timing parameters change the timing residuals when they are slightly incorrect, and then slowly build up a good timing model. To do the analysis, you will use [PINT](https://nanograv-pint.readthedocs.io/en/latest/), a Python package to time pulsars and adjust timing models. Along the way, you'll determine some interesting astrophysics about your pulsar.

# Part 0: Loading the model and the observations

First, let's import some useful packages

In [2]:
from astropy import units as u, constants as c
import astropy.time
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import copy
import io

from pint.residuals import Residuals
import pint.fitter
from pint.models import get_model
import pint.derived_quantities
import pint.simulation

We're going to use the functions defined below just for ease. You won't have to adjust them for any part of this exercise but can if you'd like to at any point in time.


- `par_writer()` will write out a pulsar parameter (par) file. It will include a couple of basic lines at top and then will simply write whatever string you want to the file. We'll see more about this in a moment. By default, it will output a par file with the name "mytiming.par".


- `make_toas()` will take a par file and simulate TOAs into a (tim) file. You can open the tim file if you'd like but it won't be required for this exercise. By default, it reads from a par file called "mytiming.par" and then will output a tim file with the name "mytiming.tim". You can see internally how PINT will simulate TOAs.

In [1]:
# Helper functions
def par_writer(string,basename='./pars/mytiming'): # can set 'basename' when calling par_writer change the file name
    # first, a few basic lines at the top of the par file


    if 'RAJ' in string:
        output = """MODE 1
        PSRJ            J0000+0000'
        PEPOCH          50000.0
        """
    else:
        output = """MODE 1
        PSRJ            J0000+0000'
        PEPOCH          50000.0
        RAJ             19:09:00
        DECJ            -37:44:00
        """
    # 'MODE 1' tells tempo2 to take into account TOA errors while fitting
    # 'PSR J0000+0000' is the name of our fake pulsar
    # 'PEPOCH' is the date (in MJD) to which the other parameters in the par file will be referenced
    # i.e., the spin frequency we give is the spin frequency on MJD 50000.0

    output += string
    with open(basename+".par",'w') as FILE:
        FILE.write(output)

def make_toas(timing_model, basename="./pars/mytiming"):
#    tstart = astropy.time.Time(1995, format="jyear")
#    tstop = astropy.time.Time(2009, format="jyear")
    tstart = astropy.time.Time(53000, format="mjd")
    tstop = astropy.time.Time(56000, format="mjd")
    # this is the error on each TOA
    error = 0.1 * u.us
    # this is a guess
    Ntoa = 215
    # make the new TOAs.  Note that even though `error` is passed, the TOAs
    # start out perfect
    toas = pint.simulation.make_fake_toas_uniform(
        tstart.mjd * u.d, tstop.mjd * u.d, Ntoa, model=timing_model, obs="ARECIBO", error=error
    )
    # So we have to still add in some noise
    toas.adjust_TOAs(astropy.time.TimeDelta(np.random.normal(size=len(toas)) * error))

    toas.write_TOA_file(basename + ".tim")
    return toas

Create a timing model for a fake pulsar using the `par_writer` function we just defined.

In [3]:
# par file for our fake pulsar
par_writer("""
F0              66.66666667
""")

This will create a file called `"mytiming.par"` inside of the `pars` folder. Now we need to load it into PINT using `get_model( )`. You do it!

In [4]:
# SOME SPACE FOR YOU TO WORK
# load the model into PINT

timing_model = get_model('./pars/mytiming.par')

Create a series of fake observations (TOAs) for the pulsar

In [8]:
pint.__version__

'0.9.3'

In [7]:
toas = make_toas(timing_model)

2023-03-25 21:48:39.182 | DEBUG    | pint.toa:__init__:1327 - No pulse number flags found in the TOAs
2023-03-25 21:48:39.183 | DEBUG    | pint.toa:apply_clock_corrections:2132 - Applying clock corrections (include_gps = True, include_bipm = False)
2023-03-25 21:48:39.194 | INFO     | pint.observatory:gps_correction:217 - Applying GPS to UTC clock correction (~few nanoseconds)
2023-03-25 21:48:39.195 | INFO     | pint.observatory:_load_gps_clock:94 - Loading global GPS clock file
2023-03-25 21:48:39.198 | DEBUG    | pint.observatory.clock_file:__init__:803 - Global clock file gps2utc.clk saving kwargs={'bogus_last_correction': False, 'valid_beyond_ends': False}
2023-03-25 21:48:39.200 | DEBUG    | pint.observatory.clock_file:read_tempo2_clock_file:460 - Loading TEMPO2-format observatory clock correction file gps2utc.clk (/home/jovyan/.astropy/cache/download/url/d3c81b5766f4bfb84e65504c8a453085/contents) with bogus_last_correction=False
2023-03-25 21:48:39.222 | INFO     | pint.observat

Now let's use `PINT` to calculate the residuals and the `matplotlib` to show how well the spin period fits the TOAs.

In [9]:
# SOME SPACE FOR YOU TO WORK

rs =  Residuals()                                                 # Calculate the residuals

xt = toas.get_mjds()                                   # Dates of the observations
plt.figure()
plt.plot(xt, rs, "x")
plt.title("%s Pre-Fit Timing Residuals" % timing_model.PSR.value)
plt.xlabel("MJD")
plt.ylabel("Residual (phase)")
plt.grid()
plt.show()

SyntaxError: invalid syntax (1093314792.py, line 3)

What do you notice about the residuals? Do you think this is a reasonable fit? Why or why not?

## Part 1: Fitting the spin

Now let's see what happens when we change the spin frequency ever so slightly. Try taking the code below and adding just 10 nanohertz to the spin frequency (1 to the last digit above)

In [ ]:
# SOME SPACE FOR YOU TO WORK

par_writer("""



""")

# no need to make a new tim file, we want to use the TOAs that we made already
# load the model into PINT
timing_model = get_model("./pars/mytiming.par")

Now let's observe how the residuals change

In [ ]:
# SOME SPACE FOR YOU TO WORK




What do you notice is happening? Why is the pattern like that?

Now let's learn how to fit the parameter for the spin period. To do this, we add a "1" after the value of the parameter, which for par files means that the parameter should be fit for. A "0" means to hold that value fixed, which is the same as not having a number there at all as we've been doing.

In [ ]:
# SOME SPACE FOR YOU TO WORK

par_writer("""



""")

# no need to make a new tim file, we want to use the TOAs that we made already
# load the model into PINT
timing_model = get_model("./pars/mytiming.par")

To do the fit we run $\texttt{pint.fitter}$ and choose the fitting algorithm (in this case, GLS), like so:

In [ ]:
# SOME SPACE FOR YOU TO WORK

# Let's tell PINT to perform the fit using the Generalized Least Squares (GLS) fitter.
fit =

# Perform the actual fit
fit.fit_toas()

# You can look at the new residuals simply by using .plot() on the fit object
fit.plot()

How well did the fit do? To see what the value of the spin period is, you have to print out the value of the spin frequency, which is  $1/P$ , since that is the more fundamental quantity. To show that value, print $\texttt{.get_allparams()["F0"]}$. In general, you can print $\texttt{.get_allparams()["parameter-name"]}$. Try that below.

In [ ]:
print(fit.get_allparams()['F0'])

We can easily write down the fitted timing into a .par file using the $\texttt{.model.as_parfile()}$ attribute of the $\texttt{fitter}$ class.

In [ ]:
# This is what the .par file will look like
print(fit.model.as_parfile(include_info=False, format="tempo2"))
# Write the .par file
with open('fitted_model.par', 'w') as f:
    f.write(fit.model.as_parfile())

## Part 2: fitting the spin-down

We know that since the radiation beams of pulsars are powered by rotation, they are losing rotational energy and thus are slowing down. The spin frequency derivative,  $\dot{f}$, is written in the par file as $\texttt{F1}$. The units of $\dot{f}$ are typically hertz per second, that is, how many hertz the pulsar is slowing down every second. As you might expect, it's a very tiny number. Let's first build our new par file and make a new set of TOAs.



In [ ]:
# SOME SPACE FOR YOU TO WORK

par_writer("""



""")

# load the model into PINT
timing_model = get_model("./pars/mytiming.par")

# create the observations
toas = make_toas(timing_model)

Let's make a plot to see how the residuals look

In [ ]:
# SOME SPACE FOR YOU TO WORK



Next, now change the spin frequency derivative ever so slightly, from  $−1 \times 10^{−19}$  to  $−2 \times 10^{−19}$.

In [ ]:
# SOME SPACE FOR YOU TO WORK
par_writer("""



""")

# no need to make a new tim file, we want to use the TOAs that we made already
# load the model into PINT
timing_model = get_model("./pars/mytiming.par")

# Now let's observe how the residuals change
rs =

plt.plot(xt, rs, "x")
plt.show()

What do you notice about the shape of the residuals? When $\dot{f}$  is incorrect, do the residuals look the same as when $\dot{f}$ is correct?

Now try to fit both parameters and print the values of the parameters. First we need to tell PINT to which parameters must be fit. Remember to use a "1" after the parameter value so PINT will know it must be fit for!

In [ ]:
# Some space for you to work
par_writer("""



""")

# no need to make a new tim file, we want to use the TOAs that we made already
# load the model into PINT
timing_model = get_model("./pars/mytiming.par")

Let's tell PINT to perform the fit using the Generalized Least Squares (GLS) fitter.

In [ ]:
# SOME SPACE FOR YOU TO WORK




How do the fitted values compare to the true values? Is the fit good?

In [ ]:
# You can look at the new residuals simply by using .plot() on the fit object
fit.plot()

# Let's look at the fitted values
print(fit.get_allparams()['     '])

## Part 3: fitting the pulsar position

Great, now you are an expert timer! Let's now add in the position of the pulsar. We can find the right ascension (RA; hours:minutes:seconds) and declination (DEC; degrees:arcminutes:arcseconds) of one of NANOGrav's best pulsars (in the J2000 coordinate system) by adding to the $\texttt{.par}$ file
`RAJ    19:09:00`
and
`DECJ    -37:44:00`.

Copy the code above, make a new par and tim file, and plot the results. As usual, you should start off with a good fit.

In [ ]:
# SOME SPACE FOR YOU TO WORK
par_writer("""



""")

# load the model into PINT
timing_model = get_model("./pars/mytiming.par")

# create the observations
toas = make_toas(timing_model)

As usual, when using the right values for these parameters you should start off with a good fit:

In [ ]:
# SOME SPACE FOR YOU TO WORK




Now add 1 milliarcsecond (0.001") to the declination and see how that affects the residuals.



In [ ]:
# SOME SPACE FOR YOU TO WORK

# Some space for you to work
par_writer("""


""")

# load the model into PINT
timing_model = get_model("./pars/mytiming.par")

Now let's observe how the residuals change

What do you notice about the fit? What is the timescale of the variation? Any ideas why?



In [ ]:
# SOME SPACE FOR YOU TO WORK

plt.xlim([xt[0], xt[int(len(xt)/5)]])
plt.vlines(      , -1, 1, linestyles ="dotted", colors ="r")
plt.vlines(       , -1, 1, linestyles ="dotted", colors ="g")
plt.plot(xt, rs, "x")
plt.show()

If you would like to investigate the residuals more closely, you can use

`toas.get_mjds()`

to get the dates of the observations,

`Residuals(toas, timing_model).phase_resids`

to get the value of the residuals, and

`Residuals(toas, timing_model).get_data_error`

in case you need the error bars (which will all be the same for this exercise by construction). Feel free to use numpy and matplotlib to investigate further.

Again, let's now fit for all of the parameters. Do you get a good fit at the end?

In [ ]:
# SOME SPACE FOR YOU TO WORK
par_writer("""




""")

# no need to make a new tim file, we want to use the TOAs that we made already
# load the model into PINT
timing_model = get_model("./pars/mytiming.par")

Let's tell PINT to perform the fit using the Generalized Least Squares (GLS) fitter.

In [ ]:
# SOME SPACE FOR YOU TO WORK



Let's look at the residuals and the fitted parameters

In [ ]:
# SOME SPACE FOR YOU TO WORK

# You can look at the new residuals simply by using .plot() on the fit object
fit.plot()

# Let's look at the fitted values
print(fit.get_allparams()['    '])
print(fit.get_allparams()['    '])

## Step 3: fitting the paralax

The pulsar is at some distance from us, which produces a parallax. The units on the parallax are in milliarcseconds (mas). A pulsar with a parallax of 1 mas is at a distance of 1 kpc (kiloparsec). Let's put a pulsar at 0.5 kpc away, so it will have a parallax of 2.0 mas.

`PX     2.0`

Then let's see what happens when the pulsar is plotted with the wrong parallax, let's say 5 mas.

Instead of making two separate cells, let's condense our work into one cell. Now just call `par_writer()` with the appropriate values, then `make_toas()`, then `par_writer()` again with the slightly adjusted parallax before plotting the residuals.

In [ ]:
# SOME SPACE FOR YOU TO WORK
par_writer("""




""")

# load the model into PINT
timing_model = get_model("./pars/mytiming.par")

# create the observations
toas = make_toas(timing_model)

# Create the timing model with the slightly adjusted parallax
par_writer("""



""")

# load the model into PINT
timing_model = get_model("./pars/mytiming.par")

# Now let's observe how the residuals change





What do you notice about the residuals? What is the timescale of the variation?

In [ ]:
# Some space for you to work
plt.xlim([xt[0], xt[int(len(xt)/5)]])
plt.vlines(     , -1, 1, linestyles ="dotted", colors ="r")
plt.vlines(     , -1, 1, linestyles ="dotted", colors ="g")
plt.plot(xt, rs, "x")
plt.show()

It turns out that the "why" of this timescale is a little bit more complicated than for an incorrect pulsar position. It has to do with the fact that the pulsar emission arriving at the solar system is not a plane wave but there is some curvature to it.

Yet again, let's fit all of the parameters. Then print out the parallax value. How does the precision on this value compare to some of the others you've examined?

In [ ]:
# SOME SPACE FOR YOU TO WORK
par_writer("""




""")

# no need to make a new tim file, we want to use the TOAs that we made already
# load the model into PINT
timing_model = get_model("./pars/mytiming.par")

# Let's tell PINT to perform the fit using the Generalized Least Squares (GLS) fitter.





# You can look at the new residuals simply by using .plot() on the fit object
fit.plot()

# Let's look at the fitted values
print(fit.get_allparams()['PX'])

## Step 4: fitting the proper motion

Let's add one more set of astrometric terms, the proper motion in the RA and DEC directions. These are given in units of milliarcseconds per year (mas/yr). In the par file, we write PMRA and PMDEC. Set the true `PMRA` to 50.0 and the `PMDEC` to 0.0 just for ease. Then simulate your TOAs, make a new par file with PMRA set to 51.0, and plot the results.

What do you notice about the residuals now? What is happening to the timescale of the variation? Any ideas as to why the structure of the residuals looks the way it does?

In [ ]:
# SOME SPACE FOR YOU TO WORK
par_writer("""




""")

# load the model into PINT
timing_model = get_model("./pars/mytiming.par")

# create the observations
toas = make_toas(timing_model)

# Create the timing model with the slightly adjusted parallax
par_writer("""



""")

# load the model into PINT
timing_model = get_model("./pars/mytiming.par")

# Now let's observe how the residuals change





Once again, let's do the fit! Check to see if the values of the proper motion make sense.



In [ ]:
# SOME SPACE FOR YOU TO WORK
par_writer("""



""")

# no need to make a new tim file, we want to use the TOAs that we made already
# load the model into PINT
timing_model = get_model("./pars/mytiming.par")

# Let's tell PINT to perform the fit using the Generalized Least Squares (GLS) fitter.






# You can look at the new residuals simply by using .plot() on the fit object





# Let's look at the fitted values




## Step 5: let's do some science!

Excellent! You now have a description of this pulsar that accounts for the spin dynamics of the pulsar and the astrometry. Let's do some science with our numbers!

First, to make things easy, let's convert from $f$ and $\dot{f}$ to $P$ and $\dot{P}$. We have by definition that the spin period and frequency are related by $P = 1/f$. Using some calculus, then we have that

$$\frac{dP}{df} = -\frac{1}{f^2},$$

and therefore

$$\dot{P} = -\frac{\dot{f}}{f^2}.$$

Use the value of $P$ given above (or re-calculate it) and calculate $\dot{P}$. Note that the units of $\dot{P}$ are seconds per seconds.

**Hint** : if you wanna extract, say, the value of `F0` from the fitted timing model, you can do

`fit.get_allparams()['F0'].value`

In [ ]:
# Some space for you to work

P = 1 / )
P_dot =
print("P = " +str(P) + " s")
print("P_dot = " + str(P_dot) + ' s/s')

Let's calculate the system's rate of energy loss, given by (see Lorimer & Kramer 2005)

$$\dot{E} = 4 \pi^2 I \dot{P} P^{-3} \simeq 3.95 \times 10^{31}~\mathrm{erg~s^{-1}}\left(\frac{\dot{P}}{10^{-15}}\right)\left(\frac{P}{\mathrm{s}}\right)^{-3}$$

How does this compare to the luminosity of the Sun ($4.0 \times 10^{33}~\mathrm{erg~s^{-1}}$)?

In [ ]:
# Some space for you to work

E_dot =
print("E_dot = " + str(E_dot) + " erg/s = " + str(E_dot/(4.0e33)) + " L_sun")

Now let's calculate the surface magnetic field of the pulsar, given by

$$ B_s = 3.2 \times 10^{19}~\mathrm{G} \sqrt{P \dot{P}} \simeq 10^{12}~\mathrm{G}~\left(\frac{\dot{P}}{10^{-15}}\right)^{1/2} \left(\frac{P}{\mathrm{s}}\right)^{1/2} $$

For reference, Earth's magnetic field is usually around $\lesssim 1~\mathrm{G}$!

In [ ]:
# Some space for you to work
import math

B =
print("B = " + str(B) + " G")